In [52]:
!pip install langchain langchain-groq faiss-cpu sentence-transformers streamlit pyngrok


In [53]:
!pip install -U langchain-community

In [54]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


In [60]:
os.environ["GROQ_API_KEY"] = "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"


In [61]:
!pip install pyPDF

In [62]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# 1. Load PDF
# Please replace "/content/your_medical_report.pdf" with the actual path to your medical report PDF file.
pdf_path = "/content/Sample-filled-in-MR.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# 2. Split text into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

docs = splitter.split_documents(pages)

# Attempt to extract patient name from the first page
patient_name = "the patient" # Default
if pages:
    first_page_content = pages[0].page_content
    # Simple attempt to find a name after "Full name of patient:"
    name_marker = "Full name of patient:"
    if name_marker in first_page_content:
        name_start = first_page_content.find(name_marker) + len(name_marker)
        # Find the end of the name, assuming it's before a newline or another section
        name_end = first_page_content.find("\n", name_start)
        if name_end == -1: # If no newline, take until the end of the string
             name_end = len(first_page_content)
        patient_name = first_page_content[name_start:name_end].strip()
        # Basic cleaning - remove extra spaces or specific markers if any
        patient_name = patient_name.split('\n')[0].strip() # Take only the first line

# 3. Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Create a new FAISS index EACH TIME
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

# 5. Create retriever
retriever = vectorstore.as_retriever()

# 6. Create QA chain
llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 7. Ask your summarization question
# Incorporate the extracted patient name into the query
query = f"Provide a clear and concise summary of the medical report for {patient_name}."


result = qa_chain(query)

# Print summary
print("SUMMARY:\n", result['result'])

# Print retrieved contexts
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\nSource {i}: {doc.page_content}")

SUMMARY:
 Here is a clear and concise summary of the medical report for Mr Tan Ah Kow:

**Patient Information:** Mr Tan Ah Kow, a 55-year-old divorced and unemployed man, living with his son.

**Medical History:** 
- Hypertension and hyperlipidemia since 1990
- Suffered several strokes in 2005

**Current Condition:** 
- Incontinent, unable to bathe or use the toilet independently, but able to feed himself
- Showing behavioral and psychological symptoms secondary to Dementia
- Gradual deterioration in cognitive and physical abilities over the years

**Mental Status:** 
- Euthymic mood, no psychotic symptoms
- Oriented to person (recognizes himself and his son) but not to place (did not know he was in a hospital clinic initially)

Source 1: seen Mr Tan regularly since then, on average once or twice a year. For purposes of 
this medical report, I re-examined Mr Tan on 20 June 2015. 
 
 
 
 
 
 
 
 
 
SECTION 3: PATIENT’S MEDICAL INFORMATION 
 
 
Patient’s clinical history: 
 
Please also 

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import streamlit as st

In [65]:
!pip install PyPDF2

In [66]:
!pip install python-docx


In [67]:
import docx


In [68]:
def summarize_medical_report(text):
  summary=" Detail AI summary of the medical report goes here"
  return summary

In [69]:
with open("summarizer.py","w") as f:
  f.write("""
def summarize_medical_report(text):
    return "This is a test summary from the backend"
    """)

In [70]:
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import tempfile

st.set_page_config(page_title="Medical Report Summarizer", page_icon="🩺")

st.title("🩺 Medical Report to Patient-Friendly Summary Generator")

uploaded_file = st.file_uploader("Upload your medical report PDF", type="pdf")

if uploaded_file is not None:
    # Save the file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(uploaded_file.read())
        pdf_path = tmp_file.name

    # 1. Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # 2. Split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    docs = splitter.split_documents(pages)

    # 3. Load embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 4. Create FAISS vectorstore
    vectorstore = FAISS.from_documents(docs, embedding=embeddings)

    # 5. Create retriever
    retriever = vectorstore.as_retriever()

    # 6. Create QA Chain
    llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    st.markdown("---")
    st.write("Click the button below to generate a patient-friendly summary of your uploaded medical report.")

    if st.button("Generate Summary 📝"):
        with st.spinner("Generating summary... ⏳"):
            query = "Summarize this medical report in simple language that a patient with no medical background can understand."
            result = qa_chain(query)

        st.subheader("📄 Summary")
        st.success(result['result'])



2025-07-17 16:03:09.537 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.546 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:09.549 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [71]:
import streamlit as st
from summarizer import summarize_medical_report  # ✅ import your backend

st.title("🩺 Medical Report Summarizer")

uploaded_file = st.file_uploader("Upload a medical report", type=["pdf", "txt"])

text_input = ""

# extract text from PDF
if uploaded_file is not None:
    if uploaded_file.type == "application/pdf":
        import PyPDF2
        reader = PyPDF2.PdfReader(uploaded_file)
        for page in reader.pages:
            text_input += page.extract_text() or ""

    elif uploaded_file.type == "text/plain":
        text_input = uploaded_file.read().decode("utf-8")

    st.text_area("Extracted Text", text_input, height=300)

# 🔁 Summarize Button
if st.button("Summarize") and text_input.strip():
    with st.spinner("Generating summary..."):
        try:
            summary = summarize_medical_report(text_input)  # ✅ CALLS YOUR FUNCTION
            st.subheader("📋 Summary:")
            st.success(summary)
        except Exception as e:
            st.error(f"Error during summarization: {e}")

2025-07-17 16:03:15.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.436 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-17 16:03:15.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [72]:
from docx import Document


In [73]:
!ngrok config add-authtoken 2zHCOZruzx5jxKOf7dkcBV2tgmz_4noZLHcq6G6V28nGZwwnS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [76]:
%%writefile app.py
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import tempfile
import os

# Assuming the Groq API key is set as an environment variable
# os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY" # Make sure this is set securely elsewhere

st.set_page_config(page_title="Medical Report Summarizer", page_icon="🩺")

# Add some basic HTML/CSS styling to the title
st.markdown(
    """
    <style>
    .title-container {
        background-color: #f0f2f6; /* Light grey background */
        padding: 20px;
        border-radius: 10px;
        text-align: center;
        margin-bottom: 30px;
    }
    .app-title {
        color: #0f117a; /* Dark blue text */
        font-size: 2.5em;
        font-weight: bold;
    }
    </style>
    <div class="title-container">
        <h1 class="app-title">🩺 Medical Report to Patient-Friendly Summary Generator</h1>
    </div>
    """,
    unsafe_allow_html=True
)


uploaded_file = st.file_uploader("Upload your medical report PDF", type="pdf")

if uploaded_file is not None:
    # Save the file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(uploaded_file.read())
        pdf_path = tmp_file.name

    # 1. Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # 2. Split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    docs = splitter.split_documents(pages)

    # 3. Load embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 4. Create FAISS vectorstore
    vectorstore = FAISS.from_documents(docs, embedding=embeddings)

    # 5. Create retriever
    retriever = vectorstore.as_retriever()

    # 6. Create QA Chain
    # Ensure GROQ_API_KEY is set in the environment or securely loaded
    if "GROQ_API_KEY" not in os.environ:
         st.error("GROQ_API_KEY environment variable not set. Please set it in a previous cell or Colab secrets.")
    else:
        llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0)
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever,
            return_source_documents=True
        )

        st.markdown("---")
        st.write("Click the button below to generate a patient-friendly summary of your uploaded medical report.")

        if st.button("Generate Summary 📝"):
            with st.spinner("Generating summary... ⏳"):
                query = "Summarize this medical report in simple language that a patient with no medical background can understand."
                result = qa_chain(query)

            st.subheader("📄 Summary")
            st.success(result['result'])

    # Clean up the temporary file
    os.remove(pdf_path)

Overwriting app.py


In [77]:
# Kill previous tunnels if any
get_ipython().system_raw('kill $(lsof -t -i:8501) || true')


# Run Streamlit in the background
get_ipython().system_raw('streamlit run app.py &')

# Create public URL using ngrok
from pyngrok import ngrok
import os
# Make sure you have added your ngrok authtoken in a previous cell
# os.environ["NGROK_AUTH_TOKEN"] = "YOUR_AUTH_TOKEN" # Uncomment and replace if not set elsewhere

public_url = ngrok.connect(8501, bind_tls=True).public_url

print("Streamlit app is live at:", public_url)

Streamlit app is live at: https://f88de1a4be5a.ngrok-free.app
